In [1]:
# Imports
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import matplotlib.pyplot as plt
#!python -m spacy download en_core_web_sm

import PyPDF2

In [2]:
# Load spacy english language model
NER = spacy.load('en_core_web_sm')

In [5]:
import os
import time
startTime = time.time()

# Load all books from path
all_books = [i for i in os.scandir("C:\\Users\\sruth\\Desktop\\Work\\Git_projects\\Character Analysis of Harry Potter books using Spacy\\Jupyter notebooks") if '.pdf' in i.name]

# Opening file and Creating a pdf reader object
file = open(all_books[0], 'rb')
fileReader = PyPDF2.PdfReader(file)

# Extract text from the book
page = fileReader.pages[0]
text = page.extract_text()
for i in range(1,len(fileReader.pages)):
    page_s = fileReader.pages[i]
    text_s = page_s.extract_text()
    text +=text_s
    

# NER
book_doc = NER(text)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime/60))

Execution time in seconds: 1.4175601442654928


In [6]:
# Visualize the identified entities
displacy.render(book_doc[0:2000],style='ent',jupyter=True)

In [97]:
# Load web scraped characters data
names = pd.read_csv('characters.csv')

# Filtering names with more than 3 words
names['word_count'] = names['character'].apply(lambda x: len(x.split()))
names = names[(names.word_count==1) | (names.word_count==2)]
names.drop('word_count',inplace=True, axis=1)

# First and Last name columns from names
names['first_name'] = names['character'].apply(lambda x: x.split(' ')[0])
names['last_name'] = names['character'].apply(lambda x: x.split(' ')[0] if len(x.split(' '))==1 else x.split(' ')[1])
names.head()

,book,character,first_name,last_name
0,Harry Potter and the Chamber of Secrets,Mr Mason,Mr,Mason
1,Harry Potter and the Chamber of Secrets,Mrs Mason,Mrs,Mason
2,Harry Potter and the Chamber of Secrets,Dobby,Dobby,Dobby
3,Harry Potter and the Chamber of Secrets,Mafalda Hopkirk,Mafalda,Hopkirk
4,Harry Potter and the Chamber of Secrets,Errol,Errol,Errol


In [98]:
# Get named entity for each sentence in the book.
entity = []

# Create dataframe with sentences and entity in it.
for i in book_doc.sents:
    entity_list = [ent.text for ent in i.ents]
    entity.append({"sentence":i, "entities":entity_list})
    
entity_df = pd.DataFrame(entity)

In [99]:
# Filter out non-entities
def filter_entity(ent_list, df):
    return [ent for ent in ent_list
           if ent in list(df.character)
           or ent in list(df.first_name)
           or ent in list(df.last_name)]

# Apply filter_entity function and remove empty lists
entity_df['character_entities'] = entity_df['entities'].apply(lambda x: filter_entity(x, names))
entity_df_filtered = entity_df[entity_df['character_entities'].map(len)>0]
entity_df_filtered = entity_df_filtered.reset_index(drop=True)

In [171]:
# Identify relationships between characters
window_size = 10 # 10 sentences
relationships = []

for i in range(entity_df_filtered.index[-1]):
    end_i = min(i+10,entity_df_filtered.index[-1])
    char_list = sum((entity_df_filtered.loc[i:end_i].character_entities), [])
    
    # Removing duplicates from the list
    char_list_unique = np.unique(char_list)
    char_list_unique = list(char_list_unique)
    
    if len(char_list_unique)>1:
        for index, name in enumerate(char_list_unique[:-1]):
            nex = char_list_unique[index+1]
            text = char_list_unique[index] +' '+ nex
            text_c = nex+' '+char_list_unique[index]
            # Removing relationships where same name appears (first and last)
            if (names['character'].isin([text])).sum()==0 and (names['character'].isin([text_c])).sum()==0: 
                relationships.append({"source":name, "target":nex})
        
# Creating DataFrame from the created list
relationships_df = pd.DataFrame(relationships)
relationships_df.head() 

,source,target
0,Dursley,Harry
1,Dursley,Harry
2,Dursley,Harold
3,Harold,Harry
4,Dursley,Harold


In [173]:
# Sort values in relationships_df
relationships_df = pd.DataFrame(np.sort(relationships_df.values, axis = 1), columns = relationships_df.columns)

# Calculate weightage for each relationship
relationships_df['value'] = 1
relationships_df = relationships_df.groupby(['source', 'target'], sort=False, as_index=False).sum()

In [184]:
# Graph Analysis and Visualization
# Create a graph from a pandas dataframe
graph = nx.from_pandas_edgelist()

TypeError: from_pandas_edgelist() missing 1 required positional argument: 'df'

In [55]:
# Steps:
# Loaded data and got NER with spacy on the first book.
# Created a csv with relationship by filtering NER of character names within 5 sentences.
# Remove character names that had more than 3 words from character.csv
# If there are more than 2 characters in char_list_unique list, create relationships between each two.

# Start at 15 minutes
# Experiment with 5 or 10 sentence windows.

,sentence,entities,character_entities
0,"(HP, \t, 1, \t, -, \t, Harry, \t, Potter, \t, ...","[1, Harry\tPotter, Stone, Harry\tPotter, Stone...",[Dursley]
1,"(Mr., \t, Dursley, \t, was, \t, the, \t, direc...","[Dursley, Grunnings]",[Dursley]
2,"(Mrs., \t, Dursley, \t, was, \t, thin, \t, and...",[Dursley],[Dursley]
3,"(The, \t, Dursleys, \t, had, \t, a, \n, small,...",[Dudley],[Dudley]
4,"(Mrs., \t, Potter, \t, was, \t, Mrs., \n, Durs...","[Dursley, several\tyears, Dursley]","[Dursley, Dursley]"


In [61]:
# import PyPDF2

# # creating an object
# file = open('Jaya_Suruthi.pdf', 'rb')

# # creating a pdf reader object
# fileReader = PyPDF2.PdfReader(file)

# # print the number of pages in pdf file
# #print(len(fileReader.pages))
# page = fileReader.pages[0]
# print(page.extract_text())

# Time taken
import time
startTime = time.time()
#
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime/60))